In [153]:
import requests
import numpy as np
from collections import defaultdict
import pandas as pd

In [268]:
company_code = 'nflx'
token = 'pk_046a8af426b441f7bde02918064ac46b'
address = "https://cloud.iexapis.com/stable/stock/{}/chart?token={}".format(company_code,token)
response = requests.get(address)

print(response.status_code)


200


In [269]:
import json

def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

# jprint(response.json())

In [270]:
from datetime import datetime

prices,dates,dates_week_format = [],[],[]

for trade in response.json():
    price,date = trade['close'],trade['date']
    date = datetime.strptime(date, "%Y-%m-%d")
    
    prices.append(price)
    dates.append(date)
    
print(dates)

[datetime.datetime(2020, 3, 18, 0, 0), datetime.datetime(2020, 3, 19, 0, 0), datetime.datetime(2020, 3, 20, 0, 0), datetime.datetime(2020, 3, 23, 0, 0), datetime.datetime(2020, 3, 24, 0, 0), datetime.datetime(2020, 3, 25, 0, 0), datetime.datetime(2020, 3, 26, 0, 0), datetime.datetime(2020, 3, 27, 0, 0), datetime.datetime(2020, 3, 30, 0, 0), datetime.datetime(2020, 3, 31, 0, 0), datetime.datetime(2020, 4, 1, 0, 0), datetime.datetime(2020, 4, 2, 0, 0), datetime.datetime(2020, 4, 3, 0, 0), datetime.datetime(2020, 4, 6, 0, 0), datetime.datetime(2020, 4, 7, 0, 0), datetime.datetime(2020, 4, 8, 0, 0), datetime.datetime(2020, 4, 9, 0, 0), datetime.datetime(2020, 4, 13, 0, 0), datetime.datetime(2020, 4, 14, 0, 0), datetime.datetime(2020, 4, 15, 0, 0), datetime.datetime(2020, 4, 16, 0, 0), datetime.datetime(2020, 4, 17, 0, 0), datetime.datetime(2020, 4, 20, 0, 0)]


In [271]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=dates,
    y=prices,
    name = company_code, 
    marker = dict(
        size = 10,
        color = 'red'
    )
))

fig.update_layout(
    showlegend=True,
    title = 'share price',
    xaxis = dict(
        range=(dates[0], dates[-1]),
        constrain='domain'
    )
)


fig.show()

In [272]:
def moving_average(N,prices):
    cumsum, moving_aves = [0], []

    for i, x in enumerate(prices, 1):
        cumsum.append(cumsum[i-1] + x)
        if i>=N:
            moving_ave = (cumsum[i] - cumsum[i-N])/N
            #can do stuff with moving_ave here
            moving_aves.append(moving_ave)

    return(moving_aves)
moving_aves = moving_average(12,prices)


In [273]:
list_N = [3,6,8]
color_dict = {list_N[0]:'pink',list_N[1]:'blue',list_N[2]:'purple'}

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=dates,
    y=prices,
    name = company_code, 
    marker = dict(
        size = 10,
        color = 'red'
    )
))
for N in list_N:
    averages = []
    averages = moving_average(N,prices)
    fig.add_trace(go.Scatter(
        x=dates,
        y=averages,
        mode='lines',
        name = '{} day moving average'.format(N), 
        marker = dict(
            size = 10,
            color = color_dict[N]
        )
    ))

fig.update_layout(
    showlegend=True,
    title = 'share price',
    xaxis = dict(
        range=(dates[0], dates[-1]),
        constrain='domain'
    )
)


fig.show()

In [274]:
import datetime

def EMA(prices,dates,N):

    EMA = defaultdict(float)
    EMA_begun = False
    index_list = [] #for index corresponding to dates not used in 
    
    
    for i,date in enumerate(dates):
        days_from_start = (date-dates[0])-datetime.timedelta(days=N)
        
        k = 2/(N+1)
        
        if days_from_start >= datetime.timedelta(days=0) and EMA_begun == False: #need to change this as will never be true due to different types
            mean_previous_days = 0
            for index in index_list:
                mean_previous_days+=prices[i]
            mean_previous_days /= len(index_list)
            EMA[date] = k*prices[i] + (1-k)*mean_previous_days
            EMA_begun = True
        elif EMA_begun == True:
            
            EMA[date] = k*prices[i]+(1-k)*EMA[dates[i-1]]
        else:
            index_list.append(i)
    
    return(EMA)
EMA(prices,dates,10)

defaultdict(float,
            {datetime.datetime(2020, 3, 30, 0, 0): 370.96,
             datetime.datetime(2020, 3, 31, 0, 0): 371.7854545454545,
             datetime.datetime(2020, 4, 1, 0, 0): 370.3844628099173,
             datetime.datetime(2020, 4, 2, 0, 0): 370.3291059353869,
             datetime.datetime(2020, 4, 3, 0, 0): 368.77108667440746,
             datetime.datetime(2020, 4, 6, 0, 0): 370.80543455178787,
             datetime.datetime(2020, 4, 7, 0, 0): 371.0735373605537,
             datetime.datetime(2020, 4, 8, 0, 0): 371.0819851131803,
             datetime.datetime(2020, 4, 9, 0, 0): 371.01616963805657,
             datetime.datetime(2020, 4, 13, 0, 0): 375.6895933402281,
             datetime.datetime(2020, 4, 14, 0, 0): 382.57330364200476,
             datetime.datetime(2020, 4, 15, 0, 0): 390.60543025254935,
             datetime.datetime(2020, 4, 16, 0, 0): 399.4353520248131,
             datetime.datetime(2020, 4, 17, 0, 0): 403.71256074757434,
             

In [275]:
list_N = [3,6,12]
color_dict = {list_N[0]:'pink',list_N[1]:'blue',list_N[2]:'purple'}
EMA_N = 12

fig = go.Figure()
fig.add_trace(go.Scatter(
    mode = 'lines+markers',
    x=dates,
    y=prices,
    name = company_code, 
    
#     marker = dict(
#         size = 10,
#         color = 'red'
#     )
))
# for N in list_N:
#     averages = []
#     averages = moving_average(N,prices)
#     fig.add_trace(go.Scatter(
#         x=dates,
#         y=averages,
#         mode='lines',
#         name = '{} day moving average'.format(N), 
#         marker = dict(
#             size = 10,
#             color = color_dict[N]
#         )
#     ))
    
EMA_values = []
EMA_dates = []
EMA_dict = EMA(prices,dates,EMA_N)

for date,value in EMA_dict.items():
    EMA_dates.append(date)
    EMA_values.append(value)
fig.add_trace(go.Scatter(
    x=EMA_dates,
    y=EMA_values,
    mode='lines',
    name = '{} day EMA'.format(N), 
    marker = dict(
        size = 10,
        color = 'black'
    )
))


    
    
fig.update_layout(
    showlegend=True,
    title = 'share price',
    xaxis = dict(
        range=(dates[0], dates[-1]),
        constrain='domain'
    )
)


fig.show()